# . 씨네21 1년 전 영화정보 분석

* 데이터 수집(1위~100위)

    - POST방식 이용
    - 메인 페이지의 영화 랭킹, 이름 크롤링
    - 해당 영화 링크 접속해 영화 세부 정보 크롤링
    - pymongo DB에 알맞은 dictionary 데이터 형태로 전처리
    - value가 여러개인 값들은 list에 추가


* 데이터 저장

    - pymongo 연결, DB 생성, Collection 생성
    - insert_many를 이용해 생성한 DB의 Collection에 입력


* 데이터 분석

    - pymongo SQL Query를 이용해 특정 조건의 데이터 추출

``씨네 21 링크 : http://www.cine21.com/rank/boxoffice/domestic_content ``

## 필요한 라이브러리 import

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import re
import pymongo

## pymongo와 연결, 데이터 넣을 DB, Collection  

In [2]:
conn = pymongo.MongoClient()
cine_db = conn.cine
movie_info = cine_db.movie

## 웹크롤링 코드 작성

In [46]:
url = "http://www.cine21.com/rank/boxoffice/domestic_content"
movies_info = []
header = {}
header['genre'] = 'all'
header['period'] = '2019-04-09'
for n in range(1,6):
    header['page'] = n
    res = requests.post(url, data=header)
    soup = bs(res.content, 'html.parser')

    movies = soup.select("ul.boxoffice_list li.boxoffice_li")
    for index, movie in enumerate(movies):
        name = movie.select_one('a > div.mov_name').text
        audience = movie.select_one('div.people_num').text
        audience = int(audience.split("|")[1].replace(",",''))
        # ranking도 문자열이므로 int형으로 바꿔주기
        ranking = int(movie.select_one('a span.grade.num1').text)
        link = movie.select_one('a')['href']
        link = f"http://www.cine21.com{link}"
        # 재귀적 접속 링크 요청 시작
        res = requests.get(link)
        soup = bs(res.content, 'html.parser')
        movie_details = {}
        year = soup.select_one("div.mov_info p.sub_info span:nth-child(1)").text
        country = soup.select_one("div.mov_info p.sub_info span:nth-child(2)").text
        proper_age = soup.select_one("div.mov_info p.sub_info span:nth-child(3)")
        if proper_age is None:
            proper_age = "연령대없음"
        else:
            proper_age = proper_age.text
        genre = soup.select_one("div.mov_info p:nth-child(4) span:nth-child(1)").text
        runtime = soup.select_one("div.mov_info p:nth-child(4) span:nth-child(2)")
        if runtime is None:
            runtime = "상영시간없음"
        else:
            runtime = runtime.text
            runtime = runtime.split(':')[1].strip()
        released_date = soup.select_one("div.mov_info p:nth-child(5) span")
        if released_date is None:
            released_date = "정보없음"
        else:
            released_date = released_date.text
            released_date = released_date.split(':')[1].strip()
        directors = soup.select("div.mov_info p:nth-child(6) a")
        directors_list = []
        for director in directors:
            director = director.text
            directors_list.append(director)
        # 인덱스 -1은 'more'을 나타내므로 마지막 인덱스를 제외한 나머지만 리스트로 담기
        actors = soup.select("div.mov_info p:nth-child(7) a")[:-1]
        actors_list = []
        for actor in actors:
            actor = actor.text
            actors_list.append(actor)

        cine_reviews = soup.select_one("ul.mov_rating li:nth-child(1) span")
        if cine_reviews is None:
            cine_reviews = "평점없음"
        else:
            cine_reviews = float(cine_reviews.text)
        people_reviews = soup.select_one("ul.mov_rating li:nth-child(2) span")
        if people_reviews is None:
            people_reviews = "평점없음"
        else:
            people_reviews = float(people_reviews.text)

        movie_details['영화이름'] = name
        movie_details['관객수'] = audience
        movie_details['순위'] = ranking
        movie_details['개봉연도'] = year
        movie_details['국가'] = country
        movie_details['관람적절연령'] = proper_age
        movie_details['장르'] = genre
        movie_details['상영시간'] = runtime
        movie_details['개봉날짜'] = released_date
        movie_details['감독'] = directors_list
        movie_details['출연배우'] = actors_list
        movie_details['씨네21평점'] = cine_reviews
        movie_details['네티즌평점'] = people_reviews

        movies_info.append(movie_details)




## pymongo DB에 데이터 잘 들어가는지 확인

- Robo 3T 앱을 이용해서 잘 들어갔는지 확인

In [35]:
movie_info.insert_one({'result':'mongo'})

In [39]:
movie_info.delete_many({})

## pymongo DB에 크롤링한 데이터 입력

In [47]:
movie_info.insert_many(movies_info)

In [4]:
# 데이터가 잘 들어갔는지 간단하게 데이터 조회
docs = movie_info.find({})
for doc in docs:
    print(doc)

{'_id': ObjectId('5e8ecdf10287aaf25b95d824'), '영화이름': '어벤져스: 엔드게임', '관객수': 13779286, '순위': 1, '개봉연도': '2019', '국가': '미국', '관람적절연령': '12세 관람가', '장르': '액션, SF', '상영시간': '181분', '개봉날짜': '2019-04-24', '감독': ['안소니 루소', '조 루소'], '출연배우': ['로버트 다우니 주니어', '크리스 에반스'], '씨네21평점': 7.17, '네티즌평점': 8.5}
{'_id': ObjectId('5e8ecdf10287aaf25b95d825'), '영화이름': '겨울왕국 2', '관객수': 13748486, '순위': 2, '개봉연도': '2019', '국가': '미국', '관람적절연령': '전체 관람가', '장르': '판타지, 코미디, 어드벤처, 애니메이션, 뮤지컬, 가족', '상영시간': '103분', '개봉날짜': '2019-11-21', '감독': ['크리스 벅', '제니퍼 리'], '출연배우': ['크리스틴 벨', '이디나 멘젤'], '씨네21평점': 6.25, '네티즌평점': 8.67}
{'_id': ObjectId('5e8ecdf10287aaf25b95d826'), '영화이름': '알라딘', '관객수': 12557831, '순위': 3, '개봉연도': '2019', '국가': '미국', '관람적절연령': '전체 관람가', '장르': '상영시간 : 128분', '상영시간': '상영시간없음', '개봉날짜': '2019-05-23', '감독': ['가이 리치'], '출연배우': ['메나 마수드', '윌 스미스'], '씨네21평점': 5.8, '네티즌평점': 8.43}
{'_id': ObjectId('5e8ecdf10287aaf25b95d827'), '영화이름': '기생충', '관객수': 10289709, '순위': 4, '개봉연도': '2019', '국가': '한국', '관람적절연령': '15세 관람가', 

## SQL 쿼리를 통한 데이터 조회

In [6]:
# 특정 key값(MySQL로 치면 Schema 이름) 수정
movie_info.update_many({},{'$rename':{'개봉연도':'개봉년도'}})

In [7]:
docs = movie_info.find({}).limit(5)
for doc in docs:
    print(doc)

{'_id': ObjectId('5e8ecdf10287aaf25b95d824'), '영화이름': '어벤져스: 엔드게임', '관객수': 13779286, '순위': 1, '국가': '미국', '관람적절연령': '12세 관람가', '장르': '액션, SF', '상영시간': '181분', '개봉날짜': '2019-04-24', '감독': ['안소니 루소', '조 루소'], '출연배우': ['로버트 다우니 주니어', '크리스 에반스'], '씨네21평점': 7.17, '네티즌평점': 8.5, '개봉년도': '2019'}
{'_id': ObjectId('5e8ecdf10287aaf25b95d825'), '영화이름': '겨울왕국 2', '관객수': 13748486, '순위': 2, '국가': '미국', '관람적절연령': '전체 관람가', '장르': '판타지, 코미디, 어드벤처, 애니메이션, 뮤지컬, 가족', '상영시간': '103분', '개봉날짜': '2019-11-21', '감독': ['크리스 벅', '제니퍼 리'], '출연배우': ['크리스틴 벨', '이디나 멘젤'], '씨네21평점': 6.25, '네티즌평점': 8.67, '개봉년도': '2019'}
{'_id': ObjectId('5e8ecdf10287aaf25b95d826'), '영화이름': '알라딘', '관객수': 12557831, '순위': 3, '국가': '미국', '관람적절연령': '전체 관람가', '장르': '상영시간 : 128분', '상영시간': '상영시간없음', '개봉날짜': '2019-05-23', '감독': ['가이 리치'], '출연배우': ['메나 마수드', '윌 스미스'], '씨네21평점': 5.8, '네티즌평점': 8.43, '개봉년도': '2019'}
{'_id': ObjectId('5e8ecdf10287aaf25b95d827'), '영화이름': '기생충', '관객수': 10289709, '순위': 4, '국가': '한국', '관람적절연령': '15세 관람가', '장르': '드라마', '상영

In [9]:
docs = movie_info.find({}).sort("네티즌평점").limit(10)
for doc in docs:
    print(doc)

{'_id': ObjectId('5e8ecdf10287aaf25b95d87f'), '영화이름': '비스트', '관객수': 203049, '순위': 92, '국가': '한국', '관람적절연령': '15세 관람가', '장르': '스릴러, 범죄', '상영시간': '130분', '개봉날짜': '2019-06-26', '감독': ['이정호'], '출연배우': ['이성민', '유재명'], '씨네21평점': 5.0, '네티즌평점': 4.0, '개봉년도': '2018'}
{'_id': ObjectId('5e8ecdf10287aaf25b95d840'), '영화이름': '정직한 후보', '관객수': 1527775, '순위': 29, '국가': '한국', '관람적절연령': '12세 관람가', '장르': '코미디', '상영시간': '105분', '개봉날짜': '2020-02-12', '감독': ['장유정'], '출연배우': ['라미란', '김무열'], '씨네21평점': 5.0, '네티즌평점': 4.2, '개봉년도': '2019'}
{'_id': ObjectId('5e8ecdf10287aaf25b95d86d'), '영화이름': '고질라: 킹 오브 몬스터', '관객수': 359538, '순위': 74, '국가': '미국', '관람적절연령': '12세 관람가', '장르': '상영시간 : 132분', '상영시간': '상영시간없음', '개봉날짜': '2019-05-29', '감독': ['마이클 도허티'], '출연배우': ['밀리 바비 브라운', '베라 파미가'], '씨네21평점': 5.25, '네티즌평점': 4.4, '개봉년도': '2019'}
{'_id': ObjectId('5e8ecdf10287aaf25b95d83c'), '영화이름': '변신', '관객수': 1805622, '순위': 25, '국가': '한국', '관람적절연령': '15세 관람가', '장르': '스릴러, 공포', '상영시간': '113분', '개봉날짜': '2019-08-21', '감독': ['김홍선'], '출연배우':

In [14]:
docs = movie_info.find({'개봉년도':{'$exists':True}}).sort('순위').limit(3)
for doc in docs:
    print(doc)

{'_id': ObjectId('5e8ecdf10287aaf25b95d824'), '영화이름': '어벤져스: 엔드게임', '관객수': 13779286, '순위': 1, '국가': '미국', '관람적절연령': '12세 관람가', '장르': '액션, SF', '상영시간': '181분', '개봉날짜': '2019-04-24', '감독': ['안소니 루소', '조 루소'], '출연배우': ['로버트 다우니 주니어', '크리스 에반스'], '씨네21평점': 7.17, '네티즌평점': 8.5, '개봉년도': '2019'}
{'_id': ObjectId('5e8ecdf10287aaf25b95d825'), '영화이름': '겨울왕국 2', '관객수': 13748486, '순위': 2, '국가': '미국', '관람적절연령': '전체 관람가', '장르': '판타지, 코미디, 어드벤처, 애니메이션, 뮤지컬, 가족', '상영시간': '103분', '개봉날짜': '2019-11-21', '감독': ['크리스 벅', '제니퍼 리'], '출연배우': ['크리스틴 벨', '이디나 멘젤'], '씨네21평점': 6.25, '네티즌평점': 8.67, '개봉년도': '2019'}
{'_id': ObjectId('5e8ecdf10287aaf25b95d826'), '영화이름': '알라딘', '관객수': 12557831, '순위': 3, '국가': '미국', '관람적절연령': '전체 관람가', '장르': '상영시간 : 128분', '상영시간': '상영시간없음', '개봉날짜': '2019-05-23', '감독': ['가이 리치'], '출연배우': ['메나 마수드', '윌 스미스'], '씨네21평점': 5.8, '네티즌평점': 8.43, '개봉년도': '2019'}


In [22]:
docs = movie_info.find({'네티즌평점':{'$exists':True}}, {'_id':0, '영화이름':1,'관객수':1,'장르':1}).limit(3)
for doc in docs:
    print(doc)

{'영화이름': '어벤져스: 엔드게임', '관객수': 13779286, '장르': '액션, SF'}
{'영화이름': '겨울왕국 2', '관객수': 13748486, '장르': '판타지, 코미디, 어드벤처, 애니메이션, 뮤지컬, 가족'}
{'영화이름': '알라딘', '관객수': 12557831, '장르': '상영시간 : 128분'}
